In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as spec
import pandas as pd
from astropy.io import ascii
from astropy.table import Table, vstack
import pickle


In [3]:
# load the TCE list
tces = ascii.read("kplr_dr25_obs_tces.txt")
tces = tces.to_pandas()


In [4]:
# load the KOI list
dr25Koi = pd.read_csv("dr25_kois_archive.txt", dtype={"kepoi_name":str})

# add TCE_ID for comparison to the TCE list
TCE_ID = []
for i in range(len(dr25Koi)):
    # format the strings so the kepid part has 9 didgets with leading zeros 
    # and the planet number part has 2 didgets with leading zeros
    TCE_ID.append("%09d"%dr25Koi.kepid.loc[i] + "-%02d"%dr25Koi.koi_tce_plnt_num.loc[i])

dr25Koi["TCE_ID"] = TCE_ID


In [5]:
# look at TCEs that have score between 0.05 and 0.1
highScoreTces = tces[(tces.Score > 0.05) & (tces.Score < 0.1)]

In [6]:
highScoreTces

,TCE_ID,KIC,Disp,Score,NTL,SS,CO,EM,period,epoch,...,Rs,Ts,logg,a,Rp/Rs,a/Rs,impact,SNR_DV,Sp,Fit_Prov
250,002010152-01,2010152,FP,0.058,1,0,0,0,317.736723,205.965172,...,1.333,6050,4.080,0.83881,0.017242,153.65,0.6261,8.4,3.031,1
621,002446113-01,2446113,FP,0.073,0,0,1,0,6.717249,137.412039,...,1.683,6078,4.094,0.07572,0.014970,21.08,0.0046,44.5,604.049,1
647,002450716-01,2450716,FP,0.065,1,0,0,0,301.757633,326.525130,...,0.768,5261,4.562,0.81242,0.018174,83.27,0.8096,6.0,0.613,1
716,002570432-01,2570432,FP,0.061,0,0,1,0,1.891448,132.591166,...,0.952,5504,4.466,0.02960,0.019142,2.35,0.4282,10.7,850.433,1
755,002578869-01,2578869,FP,0.062,1,0,0,0,515.982485,365.091066,...,2.407,5314,3.792,1.37754,0.013198,169.38,0.7259,9.5,2.181,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30102,011407805-01,11407805,FP,0.078,1,0,0,0,315.424741,393.012397,...,6.065,4833,3.130,1.10544,0.021723,789.05,0.4675,7.3,14.715,1
30897,011706658-01,11706658,FP,0.054,0,1,0,0,1.407938,132.267244,...,0.741,4627,4.564,0.02218,0.032955,3.70,0.7546,29.9,458.380,1
31040,011760931-01,11760931,FP,0.096,1,0,0,0,397.729327,264.746273,...,0.973,6032,4.485,1.07770,0.015219,229.08,0.4943,7.4,0.967,1
31763,012068928-01,12068928,FP,0.068,1,0,0,0,351.785304,406.825207,...,1.053,6161,4.454,1.02214,0.022149,795.32,0.0251,7.1,1.370,1


In [7]:
# are any KOIs?
highScoreTcesKois = dr25Koi[dr25Koi['TCE_ID'].isin(highScoreTces['TCE_ID'])]
highScoreTcesKois

,kepid,kepoi_name,kepler_name,ra,ra_err,ra_str,dec,dec_err,dec_str,koi_gmag,...,koi_fpflag_ec,koi_insol,koi_insol_err1,koi_insol_err2,koi_srho,koi_srho_err1,koi_srho_err2,koi_fittype,koi_score,TCE_ID
317,6677841,K01236.01,Kepler-279 c,287.39120,0.0,19h09m33.89s,42.194832,0.0,+42d11m41.4s,13.869,...,0,84.66,39.48,-37.28,0.07669,0.43688,-0.04179,LS+MCMC,0.086,006677841-02
370,6448890,K01241.02,Kepler-56 b,293.75836,0.0,19h35m02.01s,41.871868,0.0,+41d52m18.7s,13.255,...,0,852.28,172.91,-271.66,0.03041,0.00125,-0.00740,LS+MCMC,0.098,006448890-02
567,3734868,K00805.01,NaN,287.69675,0.0,19h10m47.22s,38.886452,0.0,+38d53m11.2s,16.254,...,0,119.31,98.49,-48.58,0.23528,0.01293,-0.00767,LS+MCMC,0.053,003734868-01
1353,8219268,K02133.01,Kepler-91 b,285.67288,0.0,19h02m41.49s,44.116730,0.0,+44d07m00.2s,13.407,...,0,3095.83,823.53,-606.19,0.00678,0.02895,-0.00151,LS+MCMC,0.059,008219268-01
1369,9718066,K02287.02,Kepler-378 c,294.94858,0.0,19h39m47.66s,46.438641,0.0,+46d26m19.1s,13.383,...,0,6.94,1.86,-0.86,1.18960,7.55335,-0.83222,LS+MCMC,0.080,009718066-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7741,11135119,K08042.01,NaN,290.57559,0.0,19h22m18.14s,48.785881,0.0,+48d47m09.2s,14.298,...,0,5.43,5.46,-4.80,144.30861,23.14445,-71.08601,LS+MCMC,0.066,011135119-01
7749,7762886,K08144.01,NaN,295.03894,0.0,19h40m09.35s,43.477329,0.0,+43d28m38.4s,16.252,...,0,0.47,0.13,-0.13,0.21107,0.01145,-0.10774,LS+MCMC,0.071,007762886-02
7854,11357185,K08051.01,NaN,295.08365,0.0,19h40m20.08s,49.170158,0.0,+49d10m12.6s,15.099,...,0,3.70,3.96,-1.21,16.26358,43.51939,-15.66833,LS+MCMC,0.066,011357185-01
7858,7900114,K08148.01,NaN,294.85776,0.0,19h39m25.86s,43.677929,0.0,+43d40m40.5s,15.933,...,0,0.81,0.52,-0.22,0.49292,0.10511,-0.26572,LS+MCMC,0.097,007900114-01


In [8]:
highScoreTcesKois.koi_pdisposition

317     FALSE POSITIVE
370     FALSE POSITIVE
567     FALSE POSITIVE
1353    FALSE POSITIVE
1369         CANDIDATE
             ...      
7741    FALSE POSITIVE
7749    FALSE POSITIVE
7854    FALSE POSITIVE
7858    FALSE POSITIVE
7955         CANDIDATE
Name: koi_pdisposition, Length: 61, dtype: object

In [9]:
highScoreTcesKois.koi_score

317     0.086
370     0.098
567     0.053
1353    0.059
1369    0.080
        ...  
7741    0.066
7749    0.071
7854    0.066
7858    0.097
7955    0.073
Name: koi_score, Length: 61, dtype: float64

In [10]:
# pull out the TCEs that are not KOIs
highScoreTcesNotKois = highScoreTces[~highScoreTces['TCE_ID'].isin(dr25Koi['TCE_ID'])]
highScoreTcesNotKois

,TCE_ID,KIC,Disp,Score,NTL,SS,CO,EM,period,epoch,...,Rs,Ts,logg,a,Rp/Rs,a/Rs,impact,SNR_DV,Sp,Fit_Prov
647,002450716-01,2450716,FP,0.065,1,0,0,0,301.757633,326.525130,...,0.768,5261,4.562,0.81242,0.018174,83.27,0.8096,6.0,0.613,1
755,002578869-01,2578869,FP,0.062,1,0,0,0,515.982485,365.091066,...,2.407,5314,3.792,1.37754,0.013198,169.38,0.7259,9.5,2.181,1
2956,003848528-01,3848528,FP,0.087,1,0,0,0,497.344347,558.388633,...,1.139,6418,4.394,1.29565,0.016124,737.23,0.0366,6.6,1.175,1
4260,004383044-01,4383044,FP,0.058,1,0,0,0,225.570849,310.953908,...,1.039,6217,4.405,0.72547,0.020117,218.43,0.6839,7.6,2.745,1
5623,004932013-01,4932013,FP,0.097,1,0,0,0,127.272659,163.373937,...,1.318,5618,4.172,0.48543,0.019544,141.46,0.6901,7.9,6.579,1
7401,005429911-01,5429911,FP,0.063,1,0,0,0,168.204412,236.571877,...,3.279,5160,3.568,0.67517,0.011041,469.87,0.3256,6.3,14.981,1
9590,005962124-01,5962124,FP,0.079,1,0,0,0,519.364824,146.511898,...,0.720,4967,4.606,1.15585,0.029623,228.80,0.9281,8.4,0.212,1
10017,006063448-03,6063448,FP,0.064,1,0,0,0,0.893646,132.249397,...,1.892,6703,4.018,0.02013,0.010427,1.29,0.9004,16.5,15984.514,1
10869,006290935-02,6290935,FP,0.051,1,0,0,0,3.422508,132.710302,...,1.425,6170,4.150,0.04513,0.004892,5.84,0.5971,7.8,1294.595,1
10969,006310627-01,6310627,FP,0.084,1,0,0,0,419.245199,167.151209,...,5.042,6599,3.325,1.37212,0.016661,186.17,0.1007,5.8,22.941,1


In [21]:
list(highScoreTcesNotKois)

['TCE_ID',
 'KIC',
 'Disp',
 'Score',
 'NTL',
 'SS',
 'CO',
 'EM',
 'period',
 'epoch',
 'Expected_MES',
 'MES',
 'NTran',
 'depth',
 'duration',
 'Rp',
 'Rs',
 'Ts',
 'logg',
 'a',
 'Rp/Rs',
 'a/Rs',
 'impact',
 'SNR_DV',
 'Sp',
 'Fit_Prov']

In [22]:
highScoreTcesNotKois.to_csv("highScoreTcesNotKois.csv", 
              columns=["TCE_ID", "Score", "period", "Rp", "SNR_DV"],
             index=False)


In [ ]:
print((highScoreTcesNotKois['period']).to_string(index=False))